In [11]:
#Chaotic Accompaniment
#In general: Listens to room audio, performs frequency analysis, consturcts an array
#of prominent constituent frequencies, and performs an arpeggio using these as notes

In [1]:
import pyaudio
import wave
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
from scipy import stats
import scipy.signal as signal
np.set_printoptions(threshold=np.inf)
from __future__ import print_function
import scipy.io.wavfile as wavfile
import scipy
import scipy.fftpack
import numpy as np
from matplotlib import pyplot as plt
from scipy.signal import argrelextrema
from pysine import sine
import random
from threading import Thread
import time
from queue import Queue

In [2]:
#Recording step: records 0.25s of audio for analysis
def record_snippet(window):
    CHUNK = 1000
    FORMAT = pyaudio.paInt16
    CHANNELS = 2
    RATE = 44100
    RECORD_SECONDS = window
    WAVE_OUTPUT_FILENAME = 'sample.wav'

    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
    return WAVE_OUTPUT_FILENAME

In [3]:
#FFT analysis step
def do_fft(filename):
    fs_rate, signal = wavfile.read(filename)
    l_audio = len(signal.shape)
    if l_audio == 2:
        signal = signal.sum(axis=1) / 2
    N = signal.shape[0]
    secs = N / float(fs_rate)
    Ts = 1.0/fs_rate # sampling interval in time
    t = scipy.arange(0, secs, Ts) # time vector as scipy arange field / numpy.ndarray
    FFT = abs(scipy.fft(signal))
    FFT_side = FFT[range(N//2)] # one side FFT range
    freqs = scipy.fftpack.fftfreq(signal.size, t[1]-t[0])
    fft_freqs = np.array(freqs)
    freqs_side = freqs[range(N//2)] # one side frequency range
    fft_freqs_side = np.array(freqs_side)

    freqs = freqs_side[30:]
    values = abs(FFT_side)[30:]
    return (freqs, values)

In [4]:
#Find maximum between 30 and 55Hz
#Find maximum between 55 and 110Hz
#Max between 110 and 220Hz
#Max between 220 and 440
#Between 440 and 600
#Between 600 and 880
#Between 880 and 1760
#A piano keyboard goes up to 4186, but pure tones in that range sound like a medical hearing test

def get_w(freqs, values):
    w_ns = []
    #find the first frequency
    max_amp = 0
    max_freq = 0
    f_min = 0
    f_max = 55
    for i in range(len(freqs)):
        if f_min<freqs[i]<f_max:
            if values[i] > max_amp:
                max_amp = values[i]
                max_freq = freqs[i]
    w_ns.append(max_freq)
    #find the second frequency
    max_amp = 0
    max_freq = 0
    f_min = 55
    f_max = 110
    for i in range(len(freqs)):
        if f_min<freqs[i]<f_max:
            if values[i] > max_amp:
                max_amp = values[i]
                max_freq = freqs[i]
    w_ns.append(max_freq)
    #find the third frequency
    max_amp = 0
    max_freq = 0
    f_min = 110
    f_max = 220
    for i in range(len(freqs)):
        if f_min<freqs[i]<f_max:
            if values[i] > max_amp:
                max_amp = values[i]
                max_freq = freqs[i]
    w_ns.append(max_freq)
    #find the fourth frequency
    max_amp = 0
    max_freq = 0
    f_min = 220
    f_max = 440
    for i in range(len(freqs)):
        if f_min<freqs[i]<f_max:
            if values[i] > max_amp:
                max_amp = values[i]
                max_freq = freqs[i]
    w_ns.append(max_freq)
    #find the fifth frequency
    max_amp = 0
    max_freq = 0
    f_min = 440
    f_max = 600
    for i in range(len(freqs)):
        if f_min<freqs[i]<f_max:
            if values[i] > max_amp:
                max_amp = values[i]
                max_freq = freqs[i]
    w_ns.append(max_freq)
    #find the sixth frequency
    max_amp = 0
    max_freq = 0
    f_min = 600
    f_max = 880
    for i in range(len(freqs)):
        if f_min<freqs[i]<f_max:
            if values[i] > max_amp:
                max_amp = values[i]
                max_freq = freqs[i]
    w_ns.append(max_freq)
    #find the seventh frequency
    max_amp = 0
    max_freq = 0
    f_min = 880
    f_max = 1760
    for i in range(len(freqs)):
        if f_min<freqs[i]<f_max:
            if values[i] > max_amp:
                max_amp = values[i]
                max_freq = freqs[i]
    w_ns.append(max_freq)
    #return the array
    return w_ns

In [5]:
def arpeggiate(notes,blips,bliplength):
    for i in range(blips):
        sine(frequency=notes[random.randint(0,6)], duration=bliplength)

In [6]:
def perform(cycles,blips,bliplength,window):
    j=0
    while(j<cycles):        
        record_snippet(window)
        arpeggiate(get_w(do_fft('sample.wav')[0],do_fft('sample.wav')[1]),blips,bliplength)
        j += 1

In [53]:
#Continuous performance: only recommended for use with headphones, or when audio input and output are well-separated
#To use: adjust the number of cycles, clip duration, and bliplength in the script, then run.
myqueue = Queue(10)
class ProducerThread(Thread):
    def run(self):
        global myqueue
        cycles = 20
        j = 0
        while(j<cycles):
            record_snippet(1.0)
            myqueue.put(get_w(do_fft('sample.wav')[0],do_fft('sample.wav')[1]))
            time.sleep(0.3)
            j += 1

class ConsumerThread(Thread):
    def run(self):
        global queue
        cycles = 20
        j = 0
        while(j<cycles):
            notes = myqueue.get()
            arpeggiate(notes,20,0.3)
            myqueue.task_done()
            time.sleep(0.3)
            j += 1
            
ProducerThread().start()
ConsumerThread().start()

In [7]:
#To use: perform(cycles,blips,bliplength,window)
#cycles is the number of listen / play rounds the instrument will perform
#blips is the number of notes the instrument will play per cycle
#bliplength is the duration of each note that the instrument will play
#window is the amount of time that the instrument will listen between rounds
perform(20,20,0.3,1.0)